# Gerando dados de Teste e Treino
---
**Autor(a):** Sabrina Bruni de Souza Faria

## Importando as Bibliotecas

In [1]:
import pandas as pd
import csv
from funcoesAuxiliares import *

## Leitura do Dataframe

In [15]:
# Dataframe com o cálculo do churn até determinada data #
# Teste #
dfC = pd.read_csv('Arquivos/churnResultadoTransNovo.csv', index_col=0)
dfC["Media_Intervalos"] = np.ceil(dfC["Media_Intervalos"])
dfC.head(5)

2.0

In [3]:
# Tabela de zeros e uns de todos os períodos do dataset análisado #
dftab = pd.read_csv('Arquivos/tabelaTotalTrans.csv', index_col=0)
dftab.head(5)

,1993-01-01,1993-01-08,1993-01-15,1993-01-22,1993-01-29,1993-02-05,1993-02-12,1993-02-19,1993-02-26,1993-03-05,...,1998-10-23,1998-10-30,1998-11-06,1998-11-13,1998-11-20,1998-11-27,1998-12-04,1998-12-11,1998-12-18,1998-12-25
2378,1,0,0,1,1,1,0,1,1,1,...,0,1,1,0,0,1,1,0,1,1
576,1,1,0,0,1,1,0,0,1,1,...,0,1,1,1,0,1,0,1,0,1
704,1,1,0,0,1,0,1,0,1,0,...,0,1,1,0,0,1,1,1,0,1
3818,1,1,0,0,1,1,0,0,1,1,...,0,1,1,0,0,1,1,1,0,1
1972,1,0,0,0,1,1,0,0,1,1,...,0,1,1,0,0,1,1,1,1,1


## Data alvo

In [4]:
data = '1998-10-22'
data_alvo = pd.to_datetime(data)
data_alvo

Timestamp('1998-10-22 00:00:00')

## Converter as colunas do DataFrame para datetime

In [5]:
datas_colunas = pd.to_datetime(dftab.columns)
dftab.columns = datas_colunas
datas_colunas

DatetimeIndex(['1993-01-01', '1993-01-08', '1993-01-15', '1993-01-22',
               '1993-01-29', '1993-02-05', '1993-02-12', '1993-02-19',
               '1993-02-26', '1993-03-05',
               ...
               '1998-10-23', '1998-10-30', '1998-11-06', '1998-11-13',
               '1998-11-20', '1998-11-27', '1998-12-04', '1998-12-11',
               '1998-12-18', '1998-12-25'],
              dtype='datetime64[ns]', length=313, freq=None)

## Encontrar a coluna mais próxima da data alvo

In [6]:
coluna_mais_proxima = datas_colunas[(datas_colunas >= data_alvo)].min()
coluna_mais_proxima

Timestamp('1998-10-23 00:00:00')

## Selecionar todas as colunas a partir da data mais próxima

In [7]:
colunas_selecionadas = datas_colunas[datas_colunas >= coluna_mais_proxima]
colunas_selecionadas

DatetimeIndex(['1998-10-23', '1998-10-30', '1998-11-06', '1998-11-13',
               '1998-11-20', '1998-11-27', '1998-12-04', '1998-12-11',
               '1998-12-18', '1998-12-25'],
              dtype='datetime64[ns]', freq=None)

## Filtrar o DataFrame para incluir apenas essas colunas

In [8]:
df_selecionado = dftab[colunas_selecionadas]
df_selecionado.head(5)

,1998-10-23 00:00:00,1998-10-30 00:00:00,1998-11-06 00:00:00,1998-11-13 00:00:00,1998-11-20 00:00:00,1998-11-27 00:00:00,1998-12-04 00:00:00,1998-12-11 00:00:00,1998-12-18 00:00:00,1998-12-25 00:00:00
2378,0,1,1,0,0,1,1,0,1,1
576,0,1,1,1,0,1,0,1,0,1
704,0,1,1,0,0,1,1,1,0,1
3818,0,1,1,0,0,1,1,1,0,1
1972,0,1,1,0,0,1,1,1,1,1


## Salvando a tabela de treino e teste em um arquivo

In [9]:
colunas_opostas = dftab.columns.difference(colunas_selecionadas)
df_dif = dftab[colunas_opostas]

In [10]:
df_dif.to_csv( "Arquivos/tabelaTreinoTrans.csv", index = True, header = True, quoting = csv.QUOTE_NONNUMERIC )
df_selecionado.to_csv( "Arquivos/tabelaTesteTrans.csv", index = True, header = True, quoting = csv.QUOTE_NONNUMERIC )

## Criando uma serie com "não churn" se houver pelo menos um valor 1 na linha, caso contrário "churn"

In [11]:
dfC = dfC.set_index("id")
resultado = df_selecionado.apply(lambda row: analisar_churn(row, dfC.loc[row.name, 'Media_Intervalos']), axis=1)
# Treino #
# Adiciona o resultado em uma nova série
resultado = pd.Series(resultado, name='resultado')
resultado.head(5)

2378    não churn
576     não churn
704     não churn
3818    não churn
1972    não churn
Name: resultado, dtype: object

## Juntando o cálculo de churn com o resultado das datas

In [12]:
# Teste e Treino #
df_merged = dfC.merge(resultado, left_index=True, right_index=True)
df_merged = df_merged.drop("Media_Intervalos", axis = 1)
# Renomeia o nome do índice
df_merged.index.name = 'id_cliente'
df_merged.head(5)

,churnLinear,churnExponencial_2,churnExponencial_e,churnRecente,resultado
id_cliente,,,,,
2378,0.290557,0.991526,0.990924,0.289474,não churn
576,0.398677,0.997203,0.998403,0.394737,não churn
704,0.259568,0.997093,0.998367,0.256579,não churn
3818,0.300109,0.997203,0.998403,0.296053,não churn
1972,0.371964,0.998958,0.999549,0.368421,não churn


## Salvando o dataframe de churn e resultado das datas em um arquivo CSV

In [13]:
df_merged.to_csv( "Arquivos/TesteTreinoChurnTrans.csv", index = True, header = True, quoting = csv.QUOTE_NONNUMERIC )